Для выполнения программы следует выполнить две нижеследующие ячейки. 
Первая выполняется без изменений: 

In [ ]:
# Установка зависимостей и используемого ПО: 
!git clone https://github.com/AnnaMedved/medicines-classification.git
%cd /content/medicines-classification
!pip install -r bert_model/requirements.txt
!pip install transformers
!git lfs install
!git clone https://huggingface.co/cointegrated/rubert-tiny bert_model/rubert-tiny

### Загрузка данных и получение результата
На этом этапе следует поместить проверочные данные в склонированную директорию content/medicines-classification (в корень), указать название файла в строке ниже и убедиться, что названия столбцов правильные и соответствуют новым данным: 


### Тесты преподготовки

In [5]:
# =============================== CNANGE ME ====================================
data = 'data.xlsx' # название файла + расширение 
columns = ['Правило взаимодействия (обр.)', 'Исходники (обр.)']# Список столбцов
# =============================== CNANGE ME ====================================

import pandas as pd
from bert_model.preprocessing_data import all_preprocessing
df = pd.read_excel(data)
df, features = all_preprocessing(data, columns)

import torch

from bert_model.bert_dataset import CustomDataset
from bert_model.bert_classifier import BertClassifier

num_classes = len(df['Подуровень'].unique())
classifier = BertClassifier(
        model_path='bert_model/rubert-tiny',
        tokenizer_path='bert_model/rubert-tiny',
        n_classes=num_classes,
        epochs=100,
        model_save_path='bert_model/bert_v3.pt'
)

# classifier = torch.load('bert_model/bert_v2.pt')

pr_d = (df[features[0]] + df[features[1]])
preds = [classifier.predict(r)+1 for r in pr_d]
result_df = df[columns]
result_df['predictions'] = preds
result_df.to_csv('result.csv')

  0%|          | 0/10094 [00:00<?, ?it/s]

  0%|          | 0/10094 [00:00<?, ?it/s]

  0%|          | 0/10094 [00:00<?, ?it/s]

  0%|          | 0/10094 [00:00<?, ?it/s]

Some weights of the model checkpoint at bert_model/rubert-tiny were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not ini

KeyboardInterrupt: 

Результат генерируется в корне: /medicines-classification/result.csv